In [1]:
# CONNECT "localhost:C:/Users/casa/Desktop/DB-Helper/PDVDATA.FDB" user 'SYSDBA' password 'masterkey';
import fdb
import pandas as pd
import datetime

In [2]:
excel_df = pd.read_excel('Provedores Todos.xlsm', sheet_name='Datos', index_col=0, na_values='--', skipfooter=10, usecols='A:AE')
con = fdb.connect(dsn="localhost:C:/Users/casa/Desktop/DB-Helper/PDVDATA.FDB", user='SYSDBA', password='masterkey', charset='UTF8')
cur = con.cursor()

c:\Users\casa\miniconda3\envs\test1\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [6]:
for idx, row in excel_df.iterrows():
    bars = False
    excel_df.loc[idx, 'PVENTA'] = ''
    if pd.notna(row['Código de Barras']):
        bars = True
        x = str(row['Código de Barras']).upper()
        if ',' in x:
            xx = x.split(',')
            pp_set = set()
            for x in xx:
                x = x.strip()
                cur.execute("SELECT PVENTA FROM PRODUCTOS WHERE CODIGO = ?;", (x.encode('utf-8'),))
                pp = cur.fetchone()
                #excel_df.loc[idx, 'PVENTA'] = excel_df.loc[idx, 'PVENTA'] + str(pp[0]) + ', '
                if pp : pp_set.add(pp[0])
            #if len(pp_set) == 1: 
            #    excel_df.loc[idx, 'PVENTA'] = str(pp_set[0])
            #    excel_df.loc[idx, 'Última Rev. Precios'] = datetime.date.today()
            #if len(pp_set) > 1:
            #    for i in pp_set:
            #        excel_df.loc[idx, 'PVENTA'] = excel_df.loc[idx, 'PVENTA'] + ', ' + str(i)
            #    excel_df.loc[idx, 'Última Rev. Precios'] = datetime.date.today()
            for i in pp_set:
                excel_df.loc[idx, 'PVENTA'] = excel_df.loc[idx, 'PVENTA'] + str(i) + ', '
            excel_df.loc[idx, 'Última Rev. Precios'] = datetime.date.today()
        else:
            cur.execute("SELECT PVENTA FROM PRODUCTOS WHERE CODIGO = ?;", (x.encode('utf-8'),))
            pp = cur.fetchone()
            if pp : excel_df.loc[idx, 'PVENTA'] = str(pp[0])
            excel_df.loc[idx, 'Última Rev. Precios'] = datetime.date.today()
    if pd.notna(row['Código SuperFuentes']):
        x = str(row['Código SuperFuentes']).upper()
        cur.execute("SELECT PVENTA FROM PRODUCTOS WHERE CODIGO = ?;", (x.encode('utf-8'),))
        pp = cur.fetchone()
        #pp_set = set()
        #pp_set.add(excel_df.loc[idx, 'PVENTA'])
        #if pp : pp_set.add(pp[0])
        #if len(pp_set) == 1 and bars: 
        #    excel_df.loc[idx, 'PVENTA'] = excel_df.loc[idx, 'PVENTA'] + ', ' + str(pp_set[0])
        #elif len(pp_set) == 1 and not bars:
        #    excel_df.loc[idx, 'PVENTA'] = str(pp_set[0])
        #elif len(pp_set) > 1 and bars:
        #    for i in pp_set:
        #        excel_df.loc[idx, 'PVENTA'] = excel_df.loc[idx, 'PVENTA'] + ', ' + str(i)
        #elif len(pp_set) > 1 and not bars:
        #    excel_df.loc[idx, 'PVENTA'] = ''
        #    for i in pp_set:
        #        excel_df.loc[idx, 'PVENTA'] = excel_df.loc[idx, 'PVENTA'] + str(i) + ', '
        #excel_df.loc[idx, 'PVENTA'] = str(pp[0])
        #if len(pp_set) >= 1:
        #    for i in pp_set:
        #            excel_df.loc[idx, 'PVENTA'] = str(i) + ', '
        if pp: 
            if bars:
                excel_df.loc[idx, 'PVENTA'] = excel_df.loc[idx, 'PVENTA'] + ', ' + str(pp[0])
            else:
                excel_df.loc[idx, 'PVENTA'] = str(pp[0])
        excel_df.loc[idx, 'Última Rev. Precios'] = datetime.date.today()

In [7]:
with pd.ExcelWriter('updated-prices.xlsx') as writer:
    excel_df.to_excel(writer)